In [19]:
import cv2
import numpy as np

colorization_release_v2.caffemodel is not added bcs of sixe you can download directly from 

https://github.com/dath1s/colorizor/blob/main/colorization_release_v2.caffemodel

In [ ]:
prototxt_path=r'models\colorization_deploy_v2.prototxt'
model_path=r'models\colorization_release_v2.caffemodel'
kernel_path=r'models\pts_in_hull.npy'

# image_path=r'images\man.webp'
image_path=r'images\black_n_white_lion.jpg'


In [21]:
# loading pre-trained model
net=cv2.dnn.readNetFromCaffe(prototxt_path,model_path)


In [ ]:
points= np.load(kernel_path)
# reshaping 1by1 kernel using cnn

In [23]:
# its opencv documentation so no proper explaination]
points=points.transpose().reshape(2,313,1,1)
net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]

# using LAB colour scheme ie Lightness alpha+ belta+

net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full([1,313],2.606,dtype="float32")]


In [24]:
bw_image=cv2.imread(image_path)
normalized=bw_image.astype('float32')/255.0
lab=cv2.cvtColor(normalized,cv2.COLOR_BGR2LAB)


In [25]:
resized=cv2.resize(lab,(224,224))
L = cv2.split(resized)[0]
L -= 50 # replicate to 3 channels
net.setInput(cv2.dnn.blobFromImage(L))






In [26]:

ab=net.forward()[0,:,:,:].transpose((1,2,0))
ab=cv2.resize(ab,(bw_image.shape[1],bw_image.shape[0]))
L=cv2.split(lab)[0]

In [27]:
colorized=np.concatenate((L[:,:,np.newaxis],ab),axis=2)
colorized=cv2.cvtColor(colorized,cv2.COLOR_LAB2BGR)
colorized=(255.0*colorized).astype("uint8")

cv2.imshow("BW Image",bw_image)
cv2.imshow("Colorized",colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()